# Magic Formula - Modelo de Investimento com Backtest

## Desafio:

- Testar se a regra de investimento da fórmula mágica de Joel Grenblatt funcionou no Brasil nos últimos anos.

## Passo a passo:

* Passo 1 - Importar as bibliotecas;
* Passo 2 - Baixar os dados disponibilizados;
* Passo 3 - Pegar as datas que iremos balancear a carteira;
* Passo 4 - Filtrar a liquidez;
* Passo 5 - Criar o ranking dos indicadores;
* Passo 6 - Criar as carteiras;
* Passo 7 - Calcular a rentabilidade por carteira;
* Passo 8 - Carcular a rentabilidade do modelo;
* Passo 9 - Calcular a rentabilidade do ibovespa no mesmo período;
* Passo 10 - Analisar os resultados.

In [ ]:
## Passo 1: Importando os módulos necessários

import pandas as pd
import quantstats as qs
